In [1]:
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from dataset import SupervisedDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
from utils import get_free_gpu
from PIL import Image
from IPython.display import clear_output

from models import O3N
from i3d import I3D

In [2]:
model_name = 'random_flow'
model_path = 'random_flow/model_6.pth'
FRAMES_PER_SEQUENCE = 10

datasets_path = "../datasets/"
video_path = os.path.join(datasets_path, "UCF101_frames")
features_path = os.path.join(datasets_path, "precomputed_features",  model_name)

try:
    os.mkdir(features_path)
except:
    pass

In [3]:
use_cuda = True

i3d = I3D(num_classes=400)
o3n = O3N(i3d=i3d, n_questions=6)
o3n.load_state_dict(torch.load(model_path))
model = o3n.i3d

if use_cuda:
    print('Using GPU')
    free_gpu_id = get_free_gpu()
    device = "cuda:{}".format(free_gpu_id)
    model.to(device)
else:
    print('Using CPU')
    
for param in model.parameters():
    param.requires_grad = False


Using GPU
GPU usage:
  memory.used memory.free
0     203 MiB   12003 MiB
1     256 MiB   11950 MiB
Using GPU0 with 12003.0 free MiB


In [ ]:
def extract_frames(vid_path, idxs):
    frames = []
    for idx in idxs:
        path = os.path.join(vid_path, "frame_{}.jpg".format(idx))
        frame = Image.open(path)
        frame = transforms.Resize((224, 224))(frame)
        frames.append(transforms.ToTensor()(frame))
    return torch.stack(frames)

In [ ]:
video_list = [f.name for f in os.scandir(video_path) if f.is_dir()]
starting_idx = 0

for v, video in enumerate(video_list[starting_idx:]):
    clear_output()
    print("{}_({}/{})".format(video,v+1+starting_idx, len(video_list)))
    video_folder = os.path.join(video_path, video)
    res_folder = os.path.join(features_path, video)
    #print(res_folder)
    try:
        os.mkdir(res_folder)
    except:
        pass
    frame_list = [f.name for f in os.scandir(video_folder) if f.is_file()]
    frame_count = len(frame_list)
    n_sequences = frame_count//FRAMES_PER_SEQUENCE
    n_files_in_folder = len(list(os.scandir(res_folder)))
    if n_files_in_folder>=n_sequences:
        print("skipping")
    else:
        for seq in range(n_sequences):
            start_idx = seq*FRAMES_PER_SEQUENCE + 1
            end_idx = start_idx + FRAMES_PER_SEQUENCE - 1
            idxs = np.arange(start_idx, end_idx+1)
            inputs = extract_frames(video_folder, idxs)[None]
            #print(inputs.shape)
            inputs = inputs.permute((0,2,1,3,4))
            model.forward(inputs.to(device))
            features = model.features.cpu().detach().numpy().ravel()
            file_name = "frames_{}_to_{}".format(start_idx, end_idx)
            np.savetxt(os.path.join(res_folder, file_name), features, fmt="%.5f")
        
    

v_ApplyLipstick_g01_c03_(4192/13320)


In [ ]:
video_list = [f.name for f in os.scandir(video_path) if f.is_dir()]

for v, video in enumerate(video_list):
    #clear_output()
    print("{}_({}/{})".format(video,v+1, len(video_list)))
    video_folder = os.path.join(video_path, video)
    res_folder = os.path.join(features_path, video)
    #print(res_folder)
    try:
        os.mkdir(res_folder)
    except:
        pass
    frame_list = [f.name for f in os.scandir(video_folder) if f.is_file()]
    frame_count = len(frame_list)
    n_sequences = frame_count//FRAMES_PER_SEQUENCE
    input_list = []
    for seq in range(n_sequences):
        start_idx = seq*FRAMES_PER_SEQUENCE + 1
        end_idx = start_idx + FRAMES_PER_SEQUENCE - 1
        idxs = np.arange(start_idx, end_idx+1)
        inputs = extract_frames(video_folder, idxs)
        input_list.append(inputs)
    inputs = torch.stack(input_list)   
    inputs = inputs.permute((0,2,1,3,4))
    model.forward(inputs.to(device))
    features = model.features.cpu().detach().numpy()
    print(features.shape)
    #file_name = "frames_{}_to_{}".format(start_idx, end_idx)
    #np.savetxt(os.path.join(res_folder, file_name), features, fmt="%.5f")

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train = np.load('precalculated_features.npy')
y_train = np.load('labels.npy')

X_test = np.load('precalculated_features_val1.npy')
y_test = np.load('labels_val1.npy')

model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.936
0.058905058905058906


In [6]:
import pandas as pd
y_pred = model.predict_proba(X_test)
df = pd.DataFrame(y_pred)
df.columns = list(range(22))
test = list(range(len(df)))
test = [i%100 for i in test]
df['video_id'] = test
print(len(df))
df = df.groupby('video_id').sum()
df['pred_label'] = df.idxmax(axis=1)
print(df['pred_label'])
y_pred = df['pred_label'].values
y_real = df.index.values

2886
video_id
0     13
1     13
2     13
3      2
4     13
5      2
6     13
7     13
8     13
9      9
10    13
11    15
12    13
13    13
14    13
15    17
16    13
17    13
18    13
19    13
20    17
21    17
22    13
23     9
24    13
25    13
26    13
27    13
28    13
29    13
      ..
70     9
71     9
72     9
73     9
74     9
75     9
76     9
77    13
78     9
79    13
80    13
81     9
82    13
83     9
84    13
85    13
86    13
87    13
88     9
89    13
90    13
91    13
92    15
93    13
94    13
95    13
96    13
97    13
98    13
99    13
Name: pred_label, Length: 100, dtype: int64


In [6]:
run final_model.py 

v_ApplyEyeMakeup_g01_c01
v_ApplyEyeMakeup_g01_c01
frames_91_to_100
frames_101_to_110
frames_121_to_130
frames_71_to_80
frames_31_to_40
frames_141_to_150
frames_51_to_60
frames_61_to_70
frames_21_to_30
frames_151_to_160
frames_81_to_90
frames_41_to_50
frames_131_to_140
frames_1_to_10
frames_11_to_20
frames_111_to_120
v_ApplyEyeMakeup_g01_c02
v_ApplyEyeMakeup_g01_c03
v_ApplyEyeMakeup_g01_c03
frames_91_to_100
frames_101_to_110
frames_121_to_130
frames_171_to_180
frames_71_to_80
frames_201_to_210
frames_31_to_40
frames_141_to_150
frames_51_to_60
frames_61_to_70
frames_21_to_30
frames_151_to_160
frames_211_to_220
frames_161_to_170
frames_81_to_90
frames_191_to_200
frames_221_to_230
frames_41_to_50
frames_131_to_140
frames_1_to_10
frames_241_to_250
frames_181_to_190
frames_11_to_20
frames_231_to_240
frames_111_to_120
v_ApplyEyeMakeup_g01_c04
v_ApplyEyeMakeup_g01_c05
v_ApplyEyeMakeup_g01_c05
frames_91_to_100
frames_101_to_110
frames_121_to_130
frames_171_to_180
frames_71_to_80
frames_201_to_2

IsADirectoryError: [Errno 21] Is a directory: '../datasets/adam_constrained_sampling/v_ApplyEyeMakeup_g01_c05/.ipynb_checkpoints'

In [ ]:
        input_list = []
        for seq in range(n_sequences):
            start_idx = seq*FRAMES_PER_SEQUENCE + 1
            end_idx = start_idx + FRAMES_PER_SEQUENCE - 1
            idxs = np.arange(start_idx, end_idx+1)
            inputs = extract_frames(video_folder, idxs)
            input_list.append(inputs)
    inputs = torch.stack(input_list)   
    inputs = inputs.permute((0,2,1,3,4))
    batch_size = len(inputs)
    if batch_size > 4:
        feature_list = []
        for row in inputs:
            #print(row.size())
            model.forward(row[None,:,:,:,:].to(device))
            feature_list.append(model.features)
        features = torch.squeeze(torch.stack(feature_list)).cpu().detach().numpy()
    else:
        #print(inputs.size())
        model.forward(inputs.to(device))
        features = model.features
        features = torch.squeeze(features)
        features = features.cpu().detach().numpy()
    #print(features.shape)
    file_name = "frames_{}_to_{}".format(start_idx, end_idx)
    #print(os.path.join(res_folder, file_name))
    np.savetxt(os.path.join(res_folder, file_name), features, fmt="%.5f")